# 大纲
这篇笔记主要涉及到四个方面
1. 模型构造
2. 参数管理
3. 自定义层
4. 读写文件

# 层和块
首先回顾一下 多层感知机 使用Sequential方法构造的网络


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10)) # 将 输入层 - 隐藏层 - 输出层 通过Sequential 链接在一起。
X = torch.rand(2,20) # 生成一个矩阵 2 * 20
net(X) # 前向计算

tensor([[-0.2930, -0.1645,  0.0174, -0.3973, -0.1224, -0.1972,  0.0418, -0.0820,
         -0.1930,  0.0182],
        [-0.1484, -0.1867, -0.1066, -0.2786, -0.0379, -0.1226, -0.0315, -0.0231,
         -0.1069,  0.0424]], grad_fn=<AddmmBackward0>)

# Pytorch --- Module

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP() # 实例化多层感知机的层
net(X)

tensor([[ 0.1185,  0.0124,  0.0934,  0.1295, -0.1545, -0.2100, -0.2180, -0.0600,
          0.0885,  0.0562],
        [ 0.1085,  0.0238,  0.1909,  0.0130, -0.1437, -0.2066, -0.1825, -0.0702,
          0.0052,  0.1233]], grad_fn=<AddmmBackward0>)

# nn.Sequential()是怎么实现的？

In [4]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block # _modules是一个特殊的容器，当参数传入到这个容器的时候，pytorch会自动认为这个东西是神经网络当中的一个层。

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.2693, -0.1063, -0.0858,  0.0596,  0.1992,  0.2189, -0.2238,  0.2035,
          0.0782, -0.1133],
        [ 0.3423, -0.1310, -0.0784,  0.0876,  0.1457,  0.1219, -0.2661,  0.2284,
          0.1043,  0.0037]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X) #
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X/=2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor([[0.4787, 0.4555, 0.8340, 0.5646, 0.0984, 0.8725, 0.8583, 0.2085, 0.4287,
         0.2374, 0.9525, 0.8228, 0.1279, 0.2299, 0.8118, 0.3331, 0.0433, 0.9108,
         0.9465, 0.4644],
        [0.6777, 0.2984, 0.4431, 0.5390, 0.1228, 0.7911, 0.5630, 0.9455, 0.2324,
         0.1269, 0.5150, 0.0213, 0.2741, 0.7813, 0.4960, 0.7490, 0.6989, 0.4944,
         0.3671, 0.6595],
        [0.3795, 0.6453, 0.9233, 0.4395, 0.7294, 0.9995, 0.1167, 0.4569, 0.2691,
         0.4039, 0.2984, 0.5077, 0.6481, 0.2042, 0.3834, 0.0487, 0.1255, 0.8748,
         0.4785, 0.4881],
        [0.2682, 0.4521, 0.1652, 0.3652, 0.0805, 0.2901, 0.2636, 0.1779, 0.3420,
         0.2244, 0.7577, 0.0812, 0.3160, 0.0157, 0.6461, 0.7249, 0.9319, 0.2615,
         0.3178, 0.0169],
        [0.0492, 0.3975, 0.1418, 0.7386, 0.3162, 0.8662, 0.5520, 0.9778, 0.2952,
         0.4046, 0.4751, 0.8925, 0.2058, 0.0923, 0.5856, 0.6992, 0.3536, 0.2148,
         0.5359, 0.7697],
        [0.0020, 0.4321, 0.7644, 0.0665, 0.7197, 0.5561, 0.2

tensor(-0.1190, grad_fn=<SumBackward0>)

# 混合搭配各种组合块的方法

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20,64),nn.ReLU(),
            nn.Linear(64,32),nn.ReLU()
        )
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(),nn.Linear(16,20),FixedHiddenMLP()
)
chimera(X)

# 参数管理

In [9]:
import torch
from torch import nn
net = nn.Sequential(
    nn.Linear(4,8),nn.ReLU(),
    nn.Linear(8,1)
)
X = torch.rand(size=(2,4))
net(X)

tensor([[0.3232],
        [0.2353]], grad_fn=<AddmmBackward0>)

## 参数访问 - 将每一层的参数拿出来

In [14]:
print(net[0].state_dict())
print(net[1])
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0285, -0.0015,  0.0136,  0.1205],
        [-0.4237, -0.1148, -0.0061, -0.4701],
        [ 0.3143, -0.3204,  0.3919,  0.2768],
        [-0.4206,  0.4978, -0.0646,  0.0909],
        [-0.1121, -0.2425,  0.0699,  0.4536],
        [-0.0639,  0.2748,  0.0149,  0.3290],
        [ 0.1113,  0.1409, -0.1603,  0.2750],
        [-0.4267, -0.2436,  0.2722,  0.2949]])), ('bias', tensor([ 0.3514, -0.0768,  0.0453, -0.1184, -0.3697,  0.2860, -0.2149, -0.0447]))])
ReLU()
OrderedDict([('weight', tensor([[ 0.2537, -0.1772, -0.1563,  0.1596, -0.1651,  0.1239,  0.2718, -0.2204]])), ('bias', tensor([0.2124]))])


In [11]:
print(type(net[2].bias)) # Parameter是可以优化的参数
print(net[2].bias) #他是一个tensor
print(net[2].bias.data) # 获取到他的值 就是 y = kx + b 当中 b的值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2124], requires_grad=True)
tensor([0.2124])


In [12]:
# 这里还没有进行反向计算呢，所以它是None
net[2].weight.grad == None

True

In [17]:
# 一次性访问所有参数
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])
# 解释一下第二个东西，它只写了 net[0]的东西 和 net[2]的东西，是因为net[1]是relu

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:
net.state_dict()['2.bias'].data # 拿到第二层的偏移

tensor([0.2124])

In [19]:
def block1():
    return nn.Sequential(
        nn.Linear(4,8),nn.ReLU(),
        nn.Linear(8,4),nn.ReLU()
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.1823],
        [0.1823]], grad_fn=<AddmmBackward0>)

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [23]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal) # 对net当中所有的layer 都进行init
net[0].weight.data[0],net[0].bias.data[0]

tensor([-0.0064,  0.0151,  0.0088, -0.0151]) tensor(0.)


(tensor([0.0035, 0.0097, 0.0018, 0.0049]), tensor(0.))

In [24]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
# 可以对不同的层使用不同的初始化函数
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5329,  0.0982, -0.2457,  0.6814])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [29]:
# 参数绑定
shared = nn.Linear(8,8)
net = nn.Sequential(
    nn.Linear(4,8),nn.ReLU(),
    shared,nn.ReLU(),
    shared,nn.ReLU(),
    nn.Linear(8,1)
) # 共享layer参数
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data == net[4].weight.data)

tensor([True, True, True, True, True, True, True, True])
tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])


# 自定义层

In [39]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,X):
        return X - X.mean()

net = nn.Sequential(
    CenteredLayer(),nn.ReLU()
)
net(torch.FloatTensor([1,2,3,4,5])) ,net(torch.rand(4,8)).mean() #理解网络 最简单的方式
# mean()函数对matrix也有效果

(tensor([0., 0., 0., 1., 2.]), tensor(0.0962))

In [42]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self,X):
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5,3)
dense.weight,dense(torch.randn(1,5))

(Parameter containing:
 tensor([[-0.8371, -1.5490,  0.2123],
         [-1.1814, -0.7740, -0.3956],
         [ 1.0152, -1.1315,  0.7895],
         [ 0.9769,  0.7218,  2.0614],
         [ 0.1837,  0.2139, -2.5674]], requires_grad=True),
 tensor([[0.0000, 0.0000, 1.8493]]))

In [40]:
torch.randn(5,3)

tensor([[ 0.9562, -0.5663, -0.0109],
        [ 1.9909,  1.9502, -0.8271],
        [ 0.0052,  0.0498, -0.3538],
        [ 0.1803,  0.7834, -1.2245],
        [ 0.4991, -0.5466,  0.3520]])

In [43]:
net = nn.Sequential(
    MyLinear(64,8),MyLinear(8,1)
)
net(torch.rand(2,64))

tensor([[13.1029],
        [ 2.7483]])

# 读写文件

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x,'x-file.pt')

x2 = torch.load("x-file.pt")
x2

tensor([0, 1, 2, 3])

In [2]:
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2 = torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [3]:
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 保存模型参数

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)

    def forward(self,x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

In [6]:
torch.save(net.state_dict(),'mlp.params')

In [7]:
# 我们上面已经存储了网络的参数，接下来，我们可以先创建一个网络，然后讲这些个参数直接带入到网络当中。
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])